In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.decomposition import TruncatedSVD
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.datasets import mnist

In [12]:
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(images, _), (_, _) = mnist.load_data()

# Normalize pixel values
images = images / 255.0

# Reshape images to (28, 28, 1)
images = images.reshape(images.shape + (1,))
print(images.shape)

# Define a simple neural network architecture
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(28 * 28 * 1, activation=None),
    layers.Reshape((28, 28, 1))  # Output layer with the same shape as input
])

model.compile(optimizer='adam', loss='mse')
model.fit(images, images, epochs=10, batch_size=32, validation_split=0.2)


(60000, 28, 28, 1)
Epoch 1/10
1500/1500 [==============================] - 11s 6ms/step - loss: 0.0603 - val_loss: 0.0565
Epoch 2/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0548 - val_loss: 0.0533
Epoch 3/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0524 - val_loss: 0.0503
Epoch 4/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0473 - val_loss: 0.0457
Epoch 5/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0451 - val_loss: 0.0445
Epoch 6/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0440 - val_loss: 0.0434
Epoch 7/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0434 - val_loss: 0.0428
Epoch 8/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0425 - val_loss: 0.0421
Epoch 9/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0418 - val_loss: 0.0414
Epoch 10/10
1500/1500 [==============================] - 8s 5m

In [13]:
# Get the output of the penultimate layer (adjust the layer index as needed)
last_layer_output = model.layers[-2].output

# Create a new model that outputs the penultimate layer's output
feature_extraction_model = models.Model(inputs=model.input, outputs=last_layer_output)

In [14]:
# Apply the model to obtain the higher-dimensional representation
higher_dimensional_representation_train = feature_extraction_model.predict(X_train)
higher_dimensional_representation_test = feature_extraction_model.predict(X_test)

# Standardize the data before applying SVD
scaler = StandardScaler()
higher_dimensional_representation_train_std = scaler.fit_transform(higher_dimensional_representation_train)
higher_dimensional_representation_test_std = scaler.transform(higher_dimensional_representation_test)

# Apply Singular Value Decomposition (SVD)
svd = TruncatedSVD(n_components=100)  # Adjust the number of components as needed
svd.fit(higher_dimensional_representation_train_std)

ValueError: in user code:

    File "/opt/homebrew/lib/python3.10/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/lib/python3.10/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/lib/python3.10/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/homebrew/lib/python3.10/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/opt/homebrew/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/homebrew/lib/python3.10/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index}

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.plot(svd.singular_values_[:50], marker='o', markersize=1)
plt.title("First 50 values of original data")
plt.xlabel("Singular Value Index")
plt.ylabel("Explained Values")
plt.grid(True)